# networking

In [ ]:
open rust_operators

()



In [ ]:
//// test

open testing

()



## types

In [ ]:
inl types () =
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"reqwest_wasm::Error\")>]\n#endif\ntype reqwest_Error = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"reqwest_wasm::RequestBuilder\")>]\n#endif\ntype reqwest_RequestBuilder = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"reqwest_wasm::Response\")>]\n#endif\ntype reqwest_Response = class end"

()



In [ ]:
inl types () =
    rust.types ()
    sm'.types ()
    types ()

()



## reqwest_response

In [ ]:
nominal reqwest_response = $"reqwest_Response"

()



## reqwest_error

In [ ]:
nominal reqwest_error = $"reqwest_Error"

()



## request_builder

In [ ]:
nominal request_builder = $"reqwest_RequestBuilder"

()



## request_type

In [ ]:
union request_type =
    | Get
    | Post

()



## request

In [ ]:
type request =
    {
        url : string
        request_type : request_type
        body : string
        json : bool
        auto_refresh : bool
    }

()



## new_request_get

In [ ]:
inl new_request_get (url : string) : request_builder =
    inl url = join url
    inl url = url |> sm'.to_std_string
    inl url = join url
    !\($'"reqwest_wasm::Client::builder().build().map_err(|err| err.to_string())?.get(!url)"')

()



## new_request_post

In [ ]:
inl new_request_post (url : string) : request_builder =
    inl url = join url
    inl url = url |> sm'.to_std_string
    inl url = join url
    !\($'"reqwest_wasm::Client::builder().build().map_err(|err| err.to_string())?.post(!url)"')

()



## request_send

In [ ]:
inl request_send (request : request_builder) : async.future_pin (resultm.result' reqwest_response reqwest_error) =
    inl request = join request
    !\($'"Box::pin(reqwest_wasm::RequestBuilder::send(!request))"')

()



## request_body

In [ ]:
inl request_body (body : string) (request : request_builder) : request_builder =
    inl body = body |> sm'.to_std_string
    !\($'"reqwest_wasm::RequestBuilder::body(!request, !body)"')

()



## request_header

In [ ]:
inl request_header (key : string) (value : string) (request : request_builder) : request_builder =
    inl request = join request
    inl key = key |> sm'.to_std_string
    inl value = value |> sm'.to_std_string
    !\($'"reqwest_wasm::RequestBuilder::header(!request, !key, !value)"')

()



## request_json

In [ ]:
inl request_json forall t. (obj : t) (request : request_builder) : request_builder =
    !\($'"reqwest_wasm::RequestBuilder::json(!request, &!obj)"')

()



## response_text

In [ ]:
inl response_text (response : reqwest_response) : async.future_pin (resultm.result' sm'.std_string reqwest_error) =
    !\($'"Box::pin(reqwest_wasm::Response::text(!response))"')

()



## tcp_client

In [ ]:
nominal tcp_client = $'System.Net.Sockets.TcpClient'

()



## new_tcp_client

In [ ]:
inl new_tcp_client () : tcp_client =
    $'new `tcp_client ()'

()



## ip_address

In [ ]:
nominal ip_address = $'System.Net.IPAddress'

()



## ip_address_parse

In [ ]:
inl ip_address_parse (s : string) : ip_address =
    s |> $'`ip_address.Parse'

()



## tcp_listener

In [ ]:
nominal tcp_listener = $'System.Net.Sockets.TcpListener'

()



## new_tcp_listener

In [ ]:
inl new_tcp_listener (ip_address : ip_address) (port : i32) : tcp_listener =
    $'new `tcp_listener (!ip_address, !port)'

()



## listener_start

In [ ]:
inl listener_start (listener : tcp_listener) : () =
    $'!listener.Start' ()

()



## listener_stop

In [ ]:
inl listener_stop (listener : tcp_listener) : () =
    $'!listener.Stop' ()

()



## client_connect_async

In [ ]:
inl client_connect_async
    (host : string)
    (port : i32)
    (ct : threading.cancellation_token)
    (client : tcp_client)
    : async.value_task
    =
    $'!client.ConnectAsync (!host, !port, !ct)'

()



## test_port_open

In [ ]:
inl test_port_open port : _ bool = async.new_async fun () =>
    inl ct = async.cancellation_token () |> async.let'
    inl client = new_tcp_client () |> use
    try_unit
        fun () =>
            client |> client_connect_async "127.0.0.1" port ct |> async.await_value_task_unit |> async.do
            return true
        fun ex =>
            inl ex = ex |> sm'.format_exception
            trace Verbose (fun () => $'$"test_port_open / ex: {!ex}"') _locals
            return false

()



In [ ]:
//// test

test_port_open 65536
|> async.run_with_timeout 120
|> _assert_eq (Some false)

module State = let mutable trace_state = None
type [<Struct>] US0 =
    | US0_0
    | US0_1
    | US0_2
    | US0_3
    | US0_4
and Mut0 = {mutable l0 : int64}
and Mut1 = {mutable l0 : bool}
and Mut2 = {mutable l0 : US0}
and Mut3 = {mutable l0 : (string -> unit)}
and [<Struct>] US1 =
    | US1_0 of f0_0 : int64
    | US1_1
and [<Struct>] US2 =
    | US2_0 of f0_0 : int64
    | US2_1
and [<Struct>] US3 =
    | US3_0 of f0_0 : bool
    | US3_1
and [<Struct>] US4 =
    | US4_0 of f0_0 : bool
    | US4_1 of f1_0 : exn
and [<Struct>] US5 =
    | US5_0 of f0_0 : bool
    | US5_1 of f1_0 : exn
let rec closure0 (v0 : string) () : string =
    let v1 : string = $"test_port_open / ex: {v0}"
    v1
and closure1 () () : string =
    let v0 : string = ""
    v0
and closure4 () (v0 : string) : unit =
    ()
and method2 (v0 : int64 option) : int64 option =
    v0
and method3 (v0 : int64 option) : int64 option =
    v0
and closure3 () () : struct (Mut0 * Mut1 * Mut2 * int64 option * Mut3) =
    let v0

## test_port_open_timeout

In [ ]:
inl test_port_open_timeout timeout port : _ bool = async.new_async_unit fun () =>
    test_port_open port
    |> async.run_with_timeout_async timeout
    |> async.let'
    |> function
        | None => false
        | Some result => result
    |> return

()



In [ ]:
//// test

test_port_open_timeout 120 65535
|> async.run_synchronously
|> _assert_eq false

module State = let mutable trace_state = None
type [<Struct>] US0 =
    | US0_0
    | US0_1
    | US0_2
    | US0_3
    | US0_4
and Mut0 = {mutable l0 : int64}
and Mut1 = {mutable l0 : bool}
and Mut2 = {mutable l0 : US0}
and Mut3 = {mutable l0 : (string -> unit)}
and [<Struct>] US1 =
    | US1_0 of f0_0 : int64
    | US1_1
and [<Struct>] US2 =
    | US2_0 of f0_0 : int64
    | US2_1
and [<Struct>] US3 =
    | US3_0 of f0_0 : bool
    | US3_1
and [<Struct>] US4 =
    | US4_0 of f0_0 : bool
    | US4_1 of f1_0 : exn
and [<Struct>] US5 =
    | US5_0 of f0_0 : bool
    | US5_1 of f1_0 : exn
let rec closure0 (v0 : string) () : string =
    let v1 : string = $"test_port_open / ex: {v0}"
    v1
and closure1 () () : string =
    let v0 : string = ""
    v0
and closure4 () (v0 : string) : unit =
    ()
and method2 (v0 : int64 option) : int64 option =
    v0
and method3 (v0 : int64 option) : int64 option =
    v0
and closure3 () () : struct (Mut0 * Mut1 * Mut2 * int64 option * Mut3) =
    let v0

## wait_for_port_access

In [ ]:
inl wait_for_port_access timeout status port : _ i64 =
    let rec loop retry : _ i64 = async.new_async_unit fun () =>
        inl isPortOpen =
            match timeout |> optionm'.unbox with
            | None => test_port_open port
            | Some timeout => test_port_open_timeout timeout port
            |> async.let'

        fix_condition
            fun () => isPortOpen = status
            fun () => retry |> return
            fun () =>
                if retry % 100 = 0 then
                    inl _locals () = $'$"port: {!port} / retry: {!retry} / timeout: %A{!timeout} / status: {!status} / {!_locals ()}"'
                    trace Verbose (fun () => "wait_for_port_access") _locals
                async.sleep 10 |> async.do
                loop (retry + 1) |> async.return_await
    loop 0i64

()



In [ ]:
//// test

inl lock_port port = async.new_async fun () =>
    trace Debug (fun () => "_1") _locals
    async.sleep 5000 |> async.do
    inl listener = new_tcp_listener ("127.0.0.1" |> ip_address_parse) port |> use
    trace Debug (fun () => "_2") _locals
    listener |> listener_start
    trace Debug (fun () => "_3") _locals
    async.sleep 2000 |> async.do
    trace Debug (fun () => "_4") _locals
    $'!listener.Stop' ()
    trace Debug (fun () => "_5") _locals

inl port = 5555i32

fun () =>
    trace Debug (fun () => "1") _locals
    inl child = lock_port port |> async.start_child |> async.let'
    trace Debug (fun () => "2") _locals
    async.sleep 1 |> async.do
    trace Debug (fun () => "3") _locals
    inl retries1 = wait_for_port_access (None |> optionm'.box) true port |> async.let'
    trace Debug (fun () => "4") _locals
    inl retries2 = wait_for_port_access (None |> optionm'.box) false port |> async.let'
    trace Debug (fun () => "5") _locals
    child |> async.do
    trace Debug (fun () => "6") _locals
    (retries1, retries2) |> return
|> async.new_async_unit
|> async.run_with_timeout 20000
|> function
    | Some (retries1, retries2) =>
        retries1
        |> _assert_between
            if runtime.is_windows () then 2i64 else 2
            if runtime.is_windows () then 5 else 1500

        retries2
        |> _assert_between
            if runtime.is_windows () then 80i64 else 80
            if runtime.is_windows () then 150 else 600

        true
    | _ => false
|> _assert_eq true

module State = let mutable trace_state = None
type [<Struct>] US0 =
    | US0_0
    | US0_1
    | US0_2
    | US0_3
    | US0_4
and Mut0 = {mutable l0 : int64}
and Mut1 = {mutable l0 : bool}
and Mut2 = {mutable l0 : US0}
and Mut3 = {mutable l0 : (string -> unit)}
and [<Struct>] US1 =
    | US1_0 of f0_0 : int64
    | US1_1
and [<Struct>] US2 =
    | US2_0 of f0_0 : int64
    | US2_1
and [<Struct>] US3 =
    | US3_0 of f0_0 : int32
    | US3_1
and [<Struct>] US4 =
    | US4_0 of f0_0 : bool
    | US4_1
and [<Struct>] US5 =
    | US5_0 of f0_0 : bool
    | US5_1 of f1_0 : exn
and [<Struct>] US6 =
    | US6_0 of f0_0 : bool
    | US6_1 of f1_0 : exn
and [<Struct>] US7 =
    | US7_0 of f0_0 : int64 * f0_1 : int64
    | US7_1
and [<Struct>] US8 =
    | US8_0 of f0_0 : int64 * f0_1 : int64
    | US8_1 of f1_0 : exn
and [<Struct>] US9 =
    | US9_0 of f0_0 : int64 * f0_1 : int64
    | US9_1 of f1_0 : exn
let rec closure0 () () : string =
    let v0 : string = "1"
    v0
and closure1 () () : s

In [ ]:
//// test

inl lock_port port = async.new_async_unit fun () =>
    trace Debug (fun () => "_1") _locals
    async.sleep 500 |> async.do
    inl listener = new_tcp_listener (ip_address_parse "127.0.0.1") port |> use
    trace Debug (fun () => "_2") _locals
    listener |> listener_start
    trace Debug (fun () => "_3") _locals
    async.sleep 200 |> async.do
    trace Debug (fun () => "_4") _locals
    listener |> listener_stop
    trace Debug (fun () => "_5") _locals

inl port = 5555

fun () =>
    trace Debug (fun () => "1") _locals
    inl child = lock_port port |> async.start_child |> async.let'
    trace Debug (fun () => "2") _locals
    async.sleep 1 |> async.do
    trace Debug (fun () => "3") _locals
    inl retries1 = wait_for_port_access (Some 60 |> optionm'.box) true port |> async.let'
    trace Debug (fun () => "4") _locals
    inl retries2 = wait_for_port_access (Some 60 |> optionm'.box) false port |> async.let'
    trace Debug (fun () => "5") _locals
    child |> async.do
    trace Debug (fun () => "6") _locals
    (retries1, retries2) |> return
|> async.new_async_unit
|> async.run_with_timeout 2000
|> function
    | Some (retries1, retries2) =>
        retries1
        |> _assert_between
            if runtime.is_windows () then 4i64 else 2
            if runtime.is_windows () then 15 else 150

        retries2
        |> _assert_between
            if runtime.is_windows () then 5i64 else 0
            if runtime.is_windows () then 20 else 60

        true
    | _ => false
|> _assert_eq true

module State = let mutable trace_state = None
type [<Struct>] US0 =
    | US0_0
    | US0_1
    | US0_2
    | US0_3
    | US0_4
and Mut0 = {mutable l0 : int64}
and Mut1 = {mutable l0 : bool}
and Mut2 = {mutable l0 : US0}
and Mut3 = {mutable l0 : (string -> unit)}
and [<Struct>] US1 =
    | US1_0 of f0_0 : int64
    | US1_1
and [<Struct>] US2 =
    | US2_0 of f0_0 : int64
    | US2_1
and [<Struct>] US3 =
    | US3_0 of f0_0 : int32
    | US3_1
and [<Struct>] US4 =
    | US4_0 of f0_0 : bool
    | US4_1
and [<Struct>] US5 =
    | US5_0 of f0_0 : bool
    | US5_1 of f1_0 : exn
and [<Struct>] US6 =
    | US6_0 of f0_0 : bool
    | US6_1 of f1_0 : exn
and [<Struct>] US7 =
    | US7_0 of f0_0 : int64 * f0_1 : int64
    | US7_1
and [<Struct>] US8 =
    | US8_0 of f0_0 : int64 * f0_1 : int64
    | US8_1 of f1_0 : exn
and [<Struct>] US9 =
    | US9_0 of f0_0 : int64 * f0_1 : int64
    | US9_1 of f1_0 : exn
let rec closure0 () () : string =
    let v0 : string = "1"
    v0
and closure1 () () : s

## get_available_port

In [ ]:
inl get_available_port timeout initial_port : _ i32 =
    let rec loop port = async.new_async_unit fun () =>
        inl is_port_open =
            match timeout |> optionm'.unbox with
            | None => test_port_open port
            | Some timeout => test_port_open_timeout timeout port
            |> async.let'
        fix_condition
            fun () => is_port_open |> not
            fun () => port |> return
            fun () => loop (port + 1) |> async.return_await
    loop initial_port

()



In [ ]:
//// test

inl lock_ports port = async.new_async_unit fun () =>
    trace Debug (fun () => "_1") _locals
    inl listener1 = new_tcp_listener (ip_address_parse "127.0.0.1") port |> use
    inl listener2 = new_tcp_listener (ip_address_parse "127.0.0.1") (port + 1) |> use
    trace Debug (fun () => "_2") _locals
    listener1 |> listener_start
    listener2 |> listener_start
    trace Debug (fun () => "_3") _locals
    async.sleep 4000 |> async.do
    trace Debug (fun () => "_4") _locals
    listener1 |> listener_stop
    listener2 |> listener_stop
    trace Debug (fun () => "_5") _locals

inl port = 5555

fun () =>
    trace Debug (fun () => "1") _locals
    inl child = lock_ports port |> async.start_child |> async.let'
    trace Debug (fun () => "2") _locals
    async.sleep 240 |> async.do
    trace Debug (fun () => "3") _locals
    inl available_port = get_available_port (None |> optionm'.box) port |> async.let'
    trace Debug (fun () => "4") _locals
    inl retries = wait_for_port_access (None |> optionm'.box) false port |> async.let'
    trace Debug (fun () => "5") _locals
    child |> async.do
    trace Debug (fun () => "6") _locals
    (available_port, retries) |> return
|> async.new_async_unit
|> async.run_with_timeout 15000
|> function
    | Some (available_port, retries) =>
        available_port |> _assert_eq (port + 2)

        retries
        |> _assert_between
            if runtime.is_windows () then 100i64 else 100
            if runtime.is_windows () then 150 else 1200

        true
    | _ => false
|> _assert_eq true

module State = let mutable trace_state = None
type [<Struct>] US0 =
    | US0_0
    | US0_1
    | US0_2
    | US0_3
    | US0_4
and Mut0 = {mutable l0 : int64}
and Mut1 = {mutable l0 : bool}
and Mut2 = {mutable l0 : US0}
and Mut3 = {mutable l0 : (string -> unit)}
and [<Struct>] US1 =
    | US1_0 of f0_0 : int64
    | US1_1
and [<Struct>] US2 =
    | US2_0 of f0_0 : int64
    | US2_1
and [<Struct>] US3 =
    | US3_0 of f0_0 : int32
    | US3_1
and [<Struct>] US4 =
    | US4_0 of f0_0 : bool
    | US4_1
and [<Struct>] US5 =
    | US5_0 of f0_0 : bool
    | US5_1 of f1_0 : exn
and [<Struct>] US6 =
    | US6_0 of f0_0 : bool
    | US6_1 of f1_0 : exn
and [<Struct>] US7 =
    | US7_0 of f0_0 : int32 * f0_1 : int64
    | US7_1
and [<Struct>] US8 =
    | US8_0 of f0_0 : int32 * f0_1 : int64
    | US8_1 of f1_0 : exn
and [<Struct>] US9 =
    | US9_0 of f0_0 : int32 * f0_1 : int64
    | US9_1 of f1_0 : exn
let rec closure0 () () : string =
    let v0 : string = "1"
    v0
and closure1 () () : s

In [ ]:
//// test

inl lock_ports port = async.new_async_unit fun () =>
    trace Debug (fun () => "_1") _locals
    inl listener1 = new_tcp_listener (ip_address_parse "127.0.0.1") port |> use
    inl listener2 = new_tcp_listener (ip_address_parse "127.0.0.1") (port + 1) |> use
    trace Debug (fun () => "_2") _locals
    listener1 |> listener_start
    listener2 |> listener_start
    trace Debug (fun () => "_3") _locals
    async.sleep 400 |> async.do
    trace Debug (fun () => "_4") _locals
    listener1 |> listener_stop
    listener2 |> listener_stop
    trace Debug (fun () => "_5") _locals

inl port = 5555

fun () =>
    trace Debug (fun () => "1") _locals
    inl child = lock_ports port |> async.start_child |> async.let'
    trace Debug (fun () => "2") _locals
    async.sleep 240 |> async.do
    trace Debug (fun () => "3") _locals
    inl availablePort = get_available_port (Some 60 |> optionm'.box) port |> async.let'
    trace Debug (fun () => "4") _locals
    inl retries = wait_for_port_access (Some 60 |> optionm'.box) false port |> async.let'
    trace Debug (fun () => "5") _locals
    child |> async.do
    trace Debug (fun () => "6") _locals
    (availablePort, retries) |> return
|> async.new_async_unit
|> async.run_with_timeout 1500
|> function
    | Some (available_port, retries) =>
        available_port |> _assert_eq (port + 2)

        retries
        |> _assert_between
            (if runtime.is_windows () then 2i64 else 1)
            (if runtime.is_windows () then 10 else 120)

        true
    | _ => false
|> _assert_eq true

module State = let mutable trace_state = None
type [<Struct>] US0 =
    | US0_0
    | US0_1
    | US0_2
    | US0_3
    | US0_4
and Mut0 = {mutable l0 : int64}
and Mut1 = {mutable l0 : bool}
and Mut2 = {mutable l0 : US0}
and Mut3 = {mutable l0 : (string -> unit)}
and [<Struct>] US1 =
    | US1_0 of f0_0 : int64
    | US1_1
and [<Struct>] US2 =
    | US2_0 of f0_0 : int64
    | US2_1
and [<Struct>] US3 =
    | US3_0 of f0_0 : int32
    | US3_1
and [<Struct>] US4 =
    | US4_0 of f0_0 : bool
    | US4_1
and [<Struct>] US5 =
    | US5_0 of f0_0 : bool
    | US5_1 of f1_0 : exn
and [<Struct>] US6 =
    | US6_0 of f0_0 : bool
    | US6_1 of f1_0 : exn
and [<Struct>] US7 =
    | US7_0 of f0_0 : int32 * f0_1 : int64
    | US7_1
and [<Struct>] US8 =
    | US8_0 of f0_0 : int32 * f0_1 : int64
    | US8_1 of f1_0 : exn
and [<Struct>] US9 =
    | US9_0 of f0_0 : int32 * f0_1 : int64
    | US9_1 of f1_0 : exn
let rec closure0 () () : string =
    let v0 : string = "1"
    v0
and closure1 () () : s

## main

In [ ]:
inl main () =
    types ()
    init_trace_state ()
    $"let test_port_open x = !test_port_open x" : ()
    $"let test_port_open_timeout x = !test_port_open_timeout x" : ()
    $"let wait_for_port_access x = !wait_for_port_access x" : ()
    $"let get_available_port x = !get_available_port x" : ()

#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func0<$0>")>]
#endif
type Func0<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func1<$0, $1>")>]
#endif
type Func0<'T, 'U> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Box<$0>")>]
#endif
type Box<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("dyn $0")>]
#endif
type Dyn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("$0 + Send")>]
#endif
type Send<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn() -> $0")>]
#endif
type Fn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn()")>]
#endif
type FnUnit = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("FnOnce() -> $0")>]
#endif
type FnOnce<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0)")>]
#endif
type ActionFn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0, $1)")>]